# Vignette B: Hierarchical network generation using SignalingProfiler

In this vignette, we are going to use SignalingProfiler:

> Venafra et al., SignalingProfiler 2.0 a network-based approach to bridge multi-omics data to phenotypic hallmarks. *npj Syst Biol Appl* **10**, 95 (2024). https://doi.org/10.1038/s41540-024-00417-6

 to build hierachical signaling models combining topology-based methods and CORNETO-CARNIVAL.

This is the python implementation of the network construction step (Step2) of SignalingProfiler workflow detailed in this [R vignette](https://html-preview.github.io/?url=https://github.com/SaccoPerfettoLab/SignalingProfiler/blob/main/SignalingProfiler-2.0-tutorial/SignalingProfiler-2.0-tutorial.html) within the package SignalingProfiler. For more information, please check the [SignalingProfiler section](../methods.html#signalingprofiler) in the Methods details and the [original SignalingProfiler paper](https://doi.org/10.1038/s41540-024-00417-6).

In [2]:
import networkcommons as nc
import decoupler as dc
import pandas as pd

SignalingProfiler offers three types of layered networks, with increasing level of deepness and computational costs. The choice is mainly based on the molecular functions of the targets proteins in your datasets. To illustrate this aspect, this notebook is showcase two applications.

## A. Generate one-layered networks from perturbed transcription factors data

In this notebook, we will use data from the NCI60 Human Tumor Cell Lines Screen. We will use the cell line 706-0. To have an overview of the cell lines, we can run `nc.data.omics.nci60_datasets()`. For more information, please check the [NCI60 details page](../datasets.html#nci60).

### A1. Input preparation

We first select Omnipath PPI netwok as a prior knowledge network.

In [ ]:
network = nc.data.network.get_omnipath()
graph = nc.utils.network_from_df(network)

Like in the [previous vignette](2_multiple_methods.ipynb), we will use a specific contrast from the [PANACEA](../datasets.html#panacea) (Afatinib versus DMSO in ASPC cell line) to extract the transcription factors that are dysregulated.

In [ ]:
dc_estimates = nc.data.omics.panacea_tables(cell_line='ASPC', drug='AFATINIB', type='TF_scores')
dc_estimates.set_index('items', inplace=True)
measurements = nc.utils.targetlayer_formatter(dc_estimates, act_col='act')

Afatinib targets EGFR by inhibition, so we will create a source dictionary containing the origin of perturbation, EGFR, and the sign of the perturbation (negative, therefore -1).

In [ ]:
source_df = pd.DataFrame({'source': ['EGFR'], 
                          'sign': [-1]}, columns=['source', 'sign'])
source_df.set_index('source', inplace=True)
sources = source_df['sign'].to_dict()

### A2. Generate one-layer network

Now, we will call SignalingProfiler to first restrict the OmniPath PPI network to all the paths of user-defined length (param ) from the source to the transcription factors. Then, running CORNETO-CARNIVAL only the sign-consistent paths will be retrieved. 
The topology-based algorithm significantly reduces the CORNETO computation time.

In [ ]:
optimized = run_signalingprofiler(sources=sources, 
                                  measurements=measurements,
                                  graph = graph, 
                                  layers = 1, 
                                  max_length=4,
                                  betaWeight=0.1, 
                                  solver='GUROBI')  

Then, we can visualized the optimized network.

In [ ]:
visualizer = nc.visual.NetworkXVisualizer(optimized)
visualizer.visualize_network(sources, measurements)

## B. Generate three-layered networks from perturbed kinases, phosphatases, and transcription factors

In this second part, we will use data modalities from the work of [Olsen et al., (2006)](https://doi.org/10.1016/j.cell.2006.09.026) where HeLa cells where stimulated with EGF treatment. 

### B1. Input preparation

We first select Omnipath PPI netwok as a prior knowledge network.

In [ ]:
network = nc.data.network.get_omnipath()
graph = nc.utils.network_from_df(network)

Since cells were stimulated with EGF, we set EGFR as a starting perturbed node for the model.

In [ ]:
source_df = pd.DataFrame({'source': ['EGFR'],
                          'sign': [-1]}, columns=['source', 'sign'])
source_df.set_index('source', inplace=True)
sources = source_df['sign'].to_dict()
sources

Using SignalingProfiler R protein activity inference pipeline, we previously inferred the activity of 56 kinases, 5 phosphatases, 3 transcription factors, and 10 other signaling proteins (see the [original publication](https://doi.org/10.1038/s41540-024-00417-6) for details). 

In [ ]:
protein_df = pd.read_table('./data/usecase/protein_activities.tsv', sep = '\t')

# Remove from proteins list the perturbed node
protein_df = protein_df.loc[protein_df['gene_name'] != 'EGFR']

# Renaming columns
protein_df_sub = protein_df[['gene_name', 'final_score']]
protein_df_sub.columns = ['items', 'act']

protein_df_sub.set_index('items', inplace=True)
measurements = nc.utils.targetlayer_formatter(protein_df_sub, act_col='act', n_elements = 100)
measurements

### B2. Generate three-layers network

Since the proteins list has kinases, phosphatases, other signaling proteins, and transcription factors, in the three-layers network SignalingProfiler will create: (i) a first layer from the perturbed receptor to kinases/phosphatases, whose length is encoded by the first argument of `max_length` parameter; (ii) a second layer with one step interactions between kinases/phosphatases and other phosphorylated proteins (not customizable); (iii) a third layer from kinases/phosphatases/other to transcription factors, whose length depends from the second argument of `max_length` parameter. Then, this network will be optimized by CARNIVAl-CORNETO to be sign-consistent with protein activities. 
The topology-based algorithm significantly reduces the CORNETO computation time.

In [ ]:
optimized = run_signalingprofiler(sources=sources, 
                                  measurements=measurements,
                                  graph = graph, 
                                  layers = 3, #also 2 performs correctly
                                  max_length=[3,4],
                                  betaWeight=0.1, 
                                  solver='GUROBI')  

Then, we can visualize the network.

In [ ]:
visualizer = nc.visual.NetworkXVisualizer(optimized)
visualizer.visualize_network(sources, measurements)